In [2]:
pip install optuna

  Using cached optuna-3.3.0-py3-none-any.whl (404 kB)
  Using cached alembic-1.12.0-py3-none-any.whl (226 kB)
  Using cached cmaes-0.10.0-py3-none-any.whl (29 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached SQLAlchemy-2.0.20-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scipy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import pandas as pd
import optuna
from optuna.trial import TrialState
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import argparse
import scipy.io
import os
import multiprocessing
import concurrent.futures

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import boto3
import io
import scipy.io

# Specify the S3 bucket and file path
bucket_name = 'sagemaker-studio-12bcc0mjktib'
file_key = '3L_ANPC_scaled_32_25.mat'

# Create a Boto3 S3 client
s3_client = boto3.client('s3')

# Download the file from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
file_content = response['Body'].read()

# Load the .mat file using scipy.io
mat_data = scipy.io.loadmat(io.BytesIO(file_content))

# Continue with the rest of your code
input_scaled = mat_data.get('input_scaled')
output_scaled = mat_data.get('output_scaled')
output_scaled = output_scaled[:,0:3]
# Split the dataset into input and output subsets
input_subset = input_scaled
output_subset = output_scaled


# Verify the shape of the resulting subsets
print(input_subset.shape)
print(output_subset.shape)


(177083, 30)
(177083, 25)


In [6]:
class MLP(nn.Module):
  def __init__(self, input_size, output_size, dropout, num_layers = 10, num_neurons = 2048):
        super(MLP, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        self.bn_layers = nn.ModuleList()
        self.activation_fn = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        for i in range(num_layers):
          if i == 0:
            self.layers.append(nn.Linear(input_size, num_neurons))
            self.bn_layers.append(nn.BatchNorm1d(num_neurons))
          else:
            self.layers.append(nn.Linear(num_neurons // (2**(i-1)), num_neurons // (2**i)))
            self.bn_layers.append(nn.BatchNorm1d(num_neurons // (2**i)))

        # Try Relu also here after going through hyperparamter tuning
        self.output_layer = nn.Linear(num_neurons // (2**(num_layers-1)), output_size)
        self.dropout = nn.Dropout(dropout)

  def forward(self, x):
      for i in range(self.num_layers):
        x = self.layers[i](x)
        x = self.bn_layers[i](x)
        x = self.dropout(x)
        x = self.activation_fn(x)

      x = self.output_layer(x)
      x = self.sigmoid(x)
      return x

In [7]:
# Set device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#Set data and target
# Remove the label column from the input data
#input_data = torch.Tensor(input_subset[:, :-1])  # Exclude the last column, assuming it contains labels
#output_data = torch.Tensor(output_subset[:, :-1])  # Include all columns, assuming it contains target values
#With label

input_data = torch.Tensor(input_subset)
output_data = torch.Tensor(output_subset)
  #print(input_data)
# Define the get_iris_data function
def get_iris_data():


  # Split the data into training, validation, and testing sets
  #test_size = 0.2 ist 80% Training set
  train_data, test_data, train_target, test_target = train_test_split(
      input_data, output_data, test_size=0.8, random_state=42
  )

  #test_size hier auf 0.5 heißt die übrigen 20% nochmal in 10% für val und 10% für test
  val_data, test_data, val_target, test_target = train_test_split(
      test_data, test_target, test_size=0.5, random_state=42
  )
  #data = pd.DataFrame(train_data.numpy())
  #print(len(data.iloc[:,7]))
  #unique_values = data.iloc[:,7].unique()
  #print(len(unique_values))
  # Create data loaders
  train_dataset = TensorDataset(train_data, train_target)
  train_loader = DataLoader(train_dataset, batch_size = 256, shuffle=True)
  val_dataset = TensorDataset(val_data, val_target)
  val_loader = DataLoader(val_dataset, batch_size = 256, shuffle=False)
  test_dataset = TensorDataset(test_data, test_target)
  test_loader = DataLoader(test_dataset, batch_size = 256, shuffle=False)

  return train_loader, val_loader, test_loader

cpu


In [8]:
# Define the dimensions of your input, hidden, and output layers
input_size = input_data.shape[1]
output_size = output_data.shape[1]

#Define inital data
test_accuracies = []
best_test_accuracy = 0.0
best_val_loss = float('inf')
train_losses = []
val_losses = []
#lr = 0.001
num_epochs = 200

# Define your data loaders here using the current batch_size
train_loader, val_loader, test_loader = get_iris_data()
# Serialize the necessary components of the data loaders
#train_loader_serialized = (train_loader.dataset,)
#val_loader_serialized = (val_loader.dataset,)

In [9]:
def train_model(trial, num_epochs, input_size, output_size, checkpoint_dir=None, model_filename=None):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize with a high value
    best_val_loss = float('inf')
    best_r2score = float('-inf')
    best_model_state_dict = None
    
    lr = trial.suggest_float("lr", 0.0001, 0.1, log=True)
    #dropout = trial.suggest_float("dropout", 0.0, 0.9)
    # Suggest a float hyperparameter with log scale and round it to 5 decimal places
    #lr = round(trial.suggest_float("lr", 0.0001, 0.1, log=True), 5)
    # Suggest a float hyperparameter and round it to 5 decimal places
    dropout = round(trial.suggest_float("dropout", 0.0, 0.5), 5)
    #num_layers = trial.suggest_int("num_layers", 1, 5)
    #num_neurons = trial.suggest_int("num_neurons", 500, 4000)
    #optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "Adamax", "RMSprop", "SGD"])
    optimizer_name = "Adam"

    # Create and compile your model using the provided hyperparameters
    #model = MLP(input_size, output_size, dropout, num_layers, num_neurons)
    model = MLP(input_size, output_size, dropout)
    model.to(device)

    criterion = nn.MSELoss()
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr)
    elif optimizer_name == "Adamax":
        optimizer = optim.Adamax(model.parameters(), lr)
    elif optimizer_name == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr)

    # Initialize a list to store the validation losses
    val_losses = []

    # Continue with the training code
    for epoch in range(num_epochs):
        train_loss = 0.0
        # Training
        model.train()
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            predictions = []
            targets = []
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                predictions.append(outputs.cpu().numpy())
                targets.append(labels.cpu().numpy())

            val_loss /= len(val_loader)
            val_losses.append(val_loss)

        predictions = np.concatenate(predictions, axis=0)
        targets = np.concatenate(targets, axis=0)
        r2score = r2_score(targets, predictions)

        # Check if the current r2score is the best so far
        if r2score > best_r2score:
            best_r2score = r2score
            best_model_state_dict = model.state_dict()
            trial.set_user_attr("model_state_dict", best_model_state_dict)
        
    return best_r2score

def objective(trial):
    #print(f"Starting trial {trial.number} on process {multiprocessing.current_process().name}")
    best_r2score= train_model(trial, num_epochs, input_size, output_size)
    return best_r2score

# Specify the S3 bucket name
bucket_name = 'sagemaker-studio-12bcc0mjktib'

# Create a Boto3 S3 client
s3_client = boto3.client('s3')

study_name = '3L_ANPC Hyperparametertuning'
study = optuna.create_study(direction="maximize", study_name=study_name)
study.optimize(objective, n_trials=50)

best_trial = study.best_trial
best_params = best_trial.params
best_metric = best_trial.value

best_model_state_dict = study.best_trial.user_attrs["model_state_dict"]
best_model = MLP(input_size, output_size, best_params["dropout"] )
#best_model = MLP(input_size, output_size, best_params["dropout"], best_params["num_layers"], best_params["num_neurons"])
best_model.to(device)
best_model.load_state_dict(best_model_state_dict)


string = "best_model_3L_ANPC_r2score_{0}_params_{1}.pt".format(
    best_metric, best_params)
    

# Save the best model after the optimization is complete
torch.save(best_model.state_dict(), string)

# Upload the best model to the S3 bucket without a folder
s3_client.upload_file(string, bucket_name, os.path.basename(string))

[I 2023-09-10 00:25:31,396] A new study created in memory with name: 2LVSI Hyperparametertuning
[I 2023-09-10 00:28:29,186] Trial 0 finished with value: 0.9465922363003625 and parameters: {'lr': 0.00015394254718630512, 'dropout': 0.3830122976118096, 'num_layers': 5, 'num_neurons': 1988}. Best is trial 0 with value: 0.9465922363003625.
[W 2023-09-10 00:31:51,761] Trial 1 failed with parameters: {'lr': 0.061949334988536296, 'dropout': 0.13825556590252613, 'num_layers': 3, 'num_neurons': 3466} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_381/3146457083.py", line 92, in objective
    best_r2score= train_model(trial, num_epochs, input_size, output_size)
  File "/tmp/ipykernel_381/3146457083.py", line 50, in train_model
    outputs = model(inputs)
  File "/opt/conda/lib/python3.10/site-packages/

KeyboardInterrupt: 